# 网格模块入门

An introduction of the mesh module in FEALPy

## 一、简介

网格是偏微分方程数值离散方法的基础，设计功能完善、易于使用的网格数据结构是一个偏微分方程数值解软件包成功的关键。FEALPy 开源软件包中的网格模块以面向对象、数组化编程的方式实现了多种常用网格的表示、存储，为各类数值方法（如 FEM、FDM）提供了标准统一的接口。因此称网格模块是 FEALPy 的核心。

### 当前支持的网格类型

|**网格名称**|**在 FEALPy 中的类名**|**分类或继承**|
|----|----|----|
|区间网格|`IntervalMesh`|非结构网格，同质网格|
|三角形网格|`TriangleMesh`|非结构网格，同质网格|
|四边形网格|`QuadrangleMesh`|非结构网格，同质网格|
|四面体网格|`TetrahedronMesh`|非结构网格，同质网格|
|六面体网格|`HexahedronMesh`|非结构网格，同质网格|
|多边形网格|`PolygonMesh`|非结构网格，非同质网格|
|一维均匀网格|`UniformMesh1d`|结构网格|
|二维均匀网格|`UniformMesh2d`|结构网格|
|三维均匀网格|`UniformMesh3d`|结构网格|
|一维单元网格|`EdgeMesh`|非结构网格，同质网格|
|二维半边网格|`HalfEdgeMesh2d`|表示半边数据结构|
|三角树网格|`Tritree`|`TriangleMesh`|
|四叉树网格|`Quadtree`|`QuadrangleMesh`|
|八叉树网格|`Octree`|`HexahedronMesh`|

### 相关术语约定

1. 网格实体

|实体名称|解释|
|----|----|
|node|节点，其拓扑维数为 0|
|edge|边，其拓扑维数为 1|
|face|面，网格中的次高维实体|
|cell|单元，网格中的最高维实体|

> 拓扑维数为 2 时，`edge` 和 `face` 指代同一类实体。

2. 常用变量

|记号|解释|
|----|----|
|`NN`|网格节点的数量|
|`NE`|网格边的数量|
|`NF`|网格面的数量|
|`NC`|网格单元的数量|
|`NVC`|网格单元的数量|
|`GD`|网格的几何维数|
|`TD`|网格的拓扑维数|


## 二、快速使用

### 创建
多数网格中提供了`from_xxx()`方法，用于快速创建网格。例如，在二维区域$$[-1, 1]\times[-1, 1]$$中创建一个三角形网格：

In [ ]:
from fealpy.mesh import TriangleMesh

mesh = TriangleMesh.from_box([-1, 1, -1, 1], nx=3, ny=3)

### 可视化
所有继承自`Plotable`的网格类均可通过`add_plot`方法实现基于 matplotlib 的可视化；也可通过`find_<entity>`系列方法，标注相应网格实体的位置和编号。

In [ ]:
from matplotlib import pyplot as plt
fig = plt.figure()
axes = fig.add_subplot()
mesh.add_plot(axes)
mesh.find_node(axes, showindex=True)
mesh.find_cell(axes, showindex=True)
plt.show()

### 获取网格实体
通过 `mesh.entity` 方法可以获得指定网格实体的数据，例如每个节点的坐标：

In [ ]:
node = mesh.entity('node')
node

又如，获得构成每个单元的节点的全局编号：

In [ ]:
cell = mesh.entity('cell')
cell

不妨与上面的图形相互印证结果。以上的 `node` 和 `cell` 数组是多数非结构网格最为基础的数据结构——存储每个单元的位置，和每个单元构成节点的全局编号。在最一般的情形下，我们通过 node 和 cell 数据来确定一个网格，如下：

In [ ]:
mesh_copy = TriangleMesh(node, cell)

此 `mesh_copy` 网格使用的 node 和 cell 数据来自前面的网格 `mesh`，因此它们是完全一样的网格（但不是同一个 Python 对象）——我们创建了一个拷贝版本。以下是部分网格在初始化中使用的数据结构总结：

|类型|基础数据结构|解释|
|----|----|----|
|非结构网格，同质网格|node, cell|节点位置，单元构成节点的编号|
|非结构网格，非同质网格|node, cell, cellLocation|节点位置，单元构成节点的编号（扁平化），每个单元在 cell 中的起始位置|
|均匀网格|extent, h, origin|范围，步长，原点位置|


## 三、常用方法

### 实体的重心和测度
1. 通过 `mesh.entity_barycenter` 方法获得网格中指定类型实体的重心的位置；
2. 通过 `mesh.entity_measure` 方法获得网格中指定实体的度量值。

### 实体计数
通过 `number_of_<entity>` 系列方法获得相应实体的数量。